# 🗺️ LIDAR 3D Height Map Scanning v13.2

## ✨ Features
| Feature | Description |
|---------|-------------|
| **Smooth Scanning** | MovL continuous motion (ไม่หยุดกระตุก) |
| **3D Height Map** | สร้าง map ความสูงทั้งพื้นที่ |
| **Multi-Pass** | สแกน 3 รอบเพื่อความแม่นยำ |
| **Camera + Map** | ใช้ภาพ + height map ร่วมกัน |

## 🎮 Workflow
1. คลิก 2 จุดกำหนด scan region
2. กด `S` เพื่อเริ่ม scan
3. ระบบสร้าง 3D height map
4. คลิกเลือก object → ใช้ height จาก map
5. กด `SPACE` เพื่อ pick

## 1️⃣ Imports

In [2]:
import cv2
import numpy as np
import time
import socket
import serial
import threading
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("✓ Imports (v13.2 - 3D Height Map)")

✓ Imports (v13.2 - 3D Height Map)


## 2️⃣ Configuration

In [3]:
# Hardware
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

# Calibration
PIXELS_PER_MM = 2.7703
ROBOT_R_OFFSET = -25.55
Z_FLOOR = -64
Z_MEASURE = 120  # Height for scanning

# LIDAR
LIDAR_PHYSICAL_OFFSET = 60
LIDAR_CORRECTION = -21
LIDAR_X_OFFSET = 25.08
LIDAR_Y_OFFSET = 20.71
HEIGHT_CORRECTION_FACTOR = 0.115

# === v13.2: Scan Parameters ===
SCAN_GRID_MM = 5         # Grid resolution: 5mm x 5mm
SCAN_PASSES = 3          # Number of scan passes
SCAN_SPEED = 10          # SpeedFactor 10%

# Gripper
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# Detection
MIN_OBJECT_AREA = 800
MAX_OBJECT_AREA = 50000

# Drop
DROP_POS = (169.71, 58.01, -17.07, 13.78)

print("✓ Configuration v13.2")
print(f"   Scan: {SCAN_GRID_MM}mm grid, {SCAN_PASSES} passes, {SCAN_SPEED}% speed")

✓ Configuration v13.2
   Scan: 5mm grid, 3 passes, 10% speed


## 3️⃣ SmartGripperController (with continuous LIDAR)

In [4]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        self._scanning = False
        self._scan_data = []  # [(x, y, z), ...]
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=0.5)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper+LIDAR on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: 
            self.serial.close()
            self.serial = None
    
    def send_command(self, cmd):
        if self.serial and self.serial.is_open:
            self.serial.reset_input_buffer()
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        self.send_command(f'G{self.mm_to_angle(grip_w)}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None
    
    def read_lidar(self, samples=5):
        if not self.serial or not self.serial.is_open:
            return None
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            start = time.time()
            while time.time() - start < 0.5:
                if self.serial.in_waiting > 0:
                    response = self.serial.readline().decode().strip()
                    if response.startswith("LIDAR:") and "ERR" not in response:
                        try:
                            readings.append(int(response.split(":")[1]))
                        except:
                            pass
                        break
            time.sleep(0.03)
        return int(np.median(readings)) if readings else None
    
    def read_lidar_fast(self):
        """Fast single LIDAR read for continuous scanning"""
        if not self.serial or not self.serial.is_open:
            return None
        self.serial.reset_input_buffer()
        self.serial.write(b'L\n')
        start = time.time()
        while time.time() - start < 0.1:
            if self.serial.in_waiting > 0:
                response = self.serial.readline().decode().strip()
                if response.startswith("LIDAR:") and "ERR" not in response:
                    try:
                        return int(response.split(":")[1])
                    except:
                        return None
        return None

print("✓ SmartGripperController")

✓ SmartGripperController


## 4️⃣ DobotControllerTCP (with MovL support)

In [5]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("SpeedFactor(50)")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        return self.send_command(f"MovJ({x},{y},{z},{r})")
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def move_linear(self, x, y, z, r=0):
        """MovL - Linear motion for smooth scanning"""
        return self.send_command(f"MovL({x},{y},{z},{r})")
    
    def set_speed(self, speed_percent):
        """Set speed factor (1-100)"""
        return self.send_command(f"SpeedFactor({speed_percent})")
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        return self.send_command(f"JointMovJ({j1},{j2},{j3},{j4})")
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ DobotControllerTCP (with MovL)")

✓ DobotControllerTCP (with MovL)


## 5️⃣ HeightMapScanner Class

In [6]:
class HeightMapScanner:
    """3D Height Map Scanner using LIDAR"""
    
    def __init__(self, robot, gripper, grid_mm=5, passes=3, speed=10):
        self.robot = robot
        self.gripper = gripper
        self.grid_mm = grid_mm
        self.passes = passes
        self.speed = speed
        self.scan_data = []  # [(x, y, z), ...]
        self.height_map = None
        self.map_bounds = None  # (x_min, x_max, y_min, y_max)
        self.grid_x = None
        self.grid_y = None
    
    def scan_region(self, x_min, x_max, y_min, y_max, z_scan=Z_MEASURE):
        """Scan a rectangular region with serpentine pattern"""
        self.scan_data = []
        self.map_bounds = (x_min, x_max, y_min, y_max)
        
        print(f"\n🗺️ Starting 3D Height Map Scan")
        print(f"   Region: X[{x_min:.0f}, {x_max:.0f}] Y[{y_min:.0f}, {y_max:.0f}]")
        print(f"   Grid: {self.grid_mm}mm | Passes: {self.passes} | Speed: {self.speed}%")
        
        # Calculate scan lines
        y_lines = np.arange(y_min, y_max + self.grid_mm, self.grid_mm)
        total_lines = len(y_lines) * self.passes
        
        # Set slow speed for scanning
        self.robot.set_speed(self.speed)
        
        for pass_num in range(self.passes):
            print(f"\n📏 Pass {pass_num + 1}/{self.passes}")
            
            for i, y in enumerate(y_lines):
                # Serpentine: alternate direction
                if (i + pass_num) % 2 == 0:
                    x_start, x_end = x_min, x_max
                else:
                    x_start, x_end = x_max, x_min
                
                # Apply LIDAR offset
                lidar_y = y + LIDAR_Y_OFFSET
                lidar_x_start = x_start + LIDAR_X_OFFSET
                lidar_x_end = x_end + LIDAR_X_OFFSET
                
                # Move to start of line
                self.robot.move_to(lidar_x_start, lidar_y, z_scan, 0)
                time.sleep(1.5)
                
                # Start continuous scan
                self._scan_line(x_start, x_end, y, z_scan)
                
                progress = ((pass_num * len(y_lines)) + i + 1) / total_lines * 100
                print(f"   Line {i+1}/{len(y_lines)} done ({progress:.0f}%)")
        
        # Restore normal speed
        self.robot.set_speed(50)
        
        # Build height map
        self._build_height_map()
        
        print(f"\n✅ Scan complete! {len(self.scan_data)} points collected")
        return self.height_map
    
    def _scan_line(self, x_start, x_end, y, z_scan):
        """Scan a single line while moving"""
        # Calculate expected scan duration
        line_length = abs(x_end - x_start)
        
        # Start linear motion (non-blocking in a way)
        lidar_x_end = x_end + LIDAR_X_OFFSET
        lidar_y = y + LIDAR_Y_OFFSET
        self.robot.move_linear(lidar_x_end, lidar_y, z_scan, 0)
        
        # Read LIDAR continuously while robot moves
        num_samples = int(line_length / self.grid_mm)
        x_step = (x_end - x_start) / max(1, num_samples)
        
        for i in range(num_samples + 1):
            x = x_start + i * x_step
            lidar_dist = self.gripper.read_lidar_fast()
            
            if lidar_dist and lidar_dist > 50 and lidar_dist < 500:
                # Convert LIDAR distance to Z height
                z_height = z_scan - lidar_dist + LIDAR_PHYSICAL_OFFSET + LIDAR_CORRECTION
                self.scan_data.append((x, y, z_height))
            
            time.sleep(0.05)  # ~20Hz sampling
        
        # Wait for motion to complete
        time.sleep(0.5)
    
    def _build_height_map(self):
        """Build interpolated height map from scan data"""
        if len(self.scan_data) < 10:
            print("❌ Not enough scan data!")
            return
        
        x_min, x_max, y_min, y_max = self.map_bounds
        
        # Create grid
        self.grid_x = np.arange(x_min, x_max + 1, self.grid_mm)
        self.grid_y = np.arange(y_min, y_max + 1, self.grid_mm)
        grid_x, grid_y = np.meshgrid(self.grid_x, self.grid_y)
        
        # Extract points
        points = np.array([(d[0], d[1]) for d in self.scan_data])
        values = np.array([d[2] for d in self.scan_data])
        
        # Interpolate
        self.height_map = griddata(points, values, (grid_x, grid_y), method='linear')
        
        # Fill NaN with floor level
        self.height_map = np.nan_to_num(self.height_map, nan=Z_FLOOR)
        
        print(f"   Height map: {self.height_map.shape} grid")
        print(f"   Z range: [{np.min(self.height_map):.1f}, {np.max(self.height_map):.1f}]")
    
    def get_height_at(self, robot_x, robot_y):
        """Get height at robot coordinates from map"""
        if self.height_map is None:
            return None
        
        x_min, x_max, y_min, y_max = self.map_bounds
        
        # Convert to grid indices
        i = int((robot_y - y_min) / self.grid_mm)
        j = int((robot_x - x_min) / self.grid_mm)
        
        # Bounds check
        if 0 <= i < self.height_map.shape[0] and 0 <= j < self.height_map.shape[1]:
            return self.height_map[i, j]
        return None
    
    def get_height_heatmap(self, frame_shape):
        """Get heatmap image for overlay on camera"""
        if self.height_map is None:
            return None
        
        # Normalize height to 0-255
        z_min, z_max = np.min(self.height_map), np.max(self.height_map)
        if z_max - z_min < 1:
            return None
        
        normalized = ((self.height_map - z_min) / (z_max - z_min) * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(normalized, cv2.COLORMAP_JET)
        
        # Resize to match frame
        heatmap = cv2.resize(heatmap, (frame_shape[1], frame_shape[0]))
        return heatmap
    
    def plot_3d(self):
        """Plot 3D surface of height map"""
        if self.height_map is None:
            print("No height map to plot!")
            return
        
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        X, Y = np.meshgrid(self.grid_x, self.grid_y)
        ax.plot_surface(X, Y, self.height_map, cmap='viridis', alpha=0.8)
        
        ax.set_xlabel('X (mm)')
        ax.set_ylabel('Y (mm)')
        ax.set_zlabel('Z (mm)')
        ax.set_title('3D Height Map')
        
        plt.show()

print("✓ HeightMapScanner class")

✓ HeightMapScanner class


## 6️⃣ ObjectDetector & GraspSelector

In [7]:
class ObjectDetectorV13:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        _, sat_mask = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY)
        _, dark_mask = cv2.threshold(v, 80, 255, cv2.THRESH_BINARY_INV)
        combined_mask = cv2.bitwise_or(sat_mask, dark_mask)
        kernel = np.ones((5, 5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if MIN_OBJECT_AREA < area < MAX_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                (cx, cy), (w, h), angle = rect
                x, y, bw, bh = cv2.boundingRect(cnt)
                objects.append({
                    'bbox': (x, y, bw, bh),
                    'center': (int(cx), int(cy)),
                    'rect': rect,
                    'contour': hull,
                    'area': cv2.contourArea(hull)
                })
        return objects

class PCAGraspSelector:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback(obj)
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]
        major = eigenvectors[:, 0]
        minor = eigenvectors[:, 1]
        angle = np.degrees(np.arctan2(major[1], major[0]))
        proj = np.dot(pts_centered, minor)
        width_mm = (np.max(proj) - np.min(proj)) / self.ppm
        cx, cy = int(mean[0]), int(mean[1])
        grasp_angle = self._normalize(angle + 90)
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            return [{'center': (cx, cy), 'width_mm': width_mm, 
                     'camera_angle': grasp_angle, 'score': 1.0, 'type': 'PCA'}]
        return self._fallback(obj)
    
    def _fallback(self, obj):
        rect = obj.get('rect')
        if not rect: return []
        (cx, cy), (w, h), angle = rect
        grip_w = min(w, h) / self.ppm
        grip_a = angle + 90 if w < h else angle
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            return [{'center': (int(cx), int(cy)), 'width_mm': grip_w,
                     'camera_angle': self._normalize(grip_a), 'score': 0.6, 'type': 'Rect'}]
        return []
    
    def _normalize(self, a):
        while a > 90: a -= 180
        while a < -90: a += 180
        return a

print("✓ ObjectDetector & GraspSelector")

✓ ObjectDetector & GraspSelector


## 7️⃣ Initialize Components

In [8]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = ObjectDetectorV13(PIXELS_PER_MM)
grasp_selector = PCAGraspSelector(PIXELS_PER_MM)
scanner = HeightMapScanner(robot, gripper, SCAN_GRID_MM, SCAN_PASSES, SCAN_SPEED)

print("✓ Components initialized (v13.2)")

✓ Components initialized (v13.2)


---
# 🤖 CONNECT ROBOT & GRIPPER
---

In [9]:
print("="*60)
print("🤖 Connecting to Robot and Gripper+LIDAR...")
print("="*60)

gripper.connect()
robot.connect(ROBOT_IP)
robot.home()

print("✅ Ready!")

🤖 Connecting to Robot and Gripper+LIDAR...
✅ Gripper+LIDAR on COM9
✅ Robot connected!
🤖 HOME...
✅ Ready!


---
# 🗺️ MAIN SYSTEM v13.2
---

In [10]:
# === Global State ===
scan_region_pixels = []  # [(x1, y1), (x2, y2)]
selected_object = None
selected_grasp = None
current_grasps = []
detected_objects = []
height_map_ready = False
show_heatmap = False

def mouse_callback(event, x, y, flags, param):
    global scan_region_pixels, selected_object, selected_grasp, current_grasps
    
    # RIGHT CLICK = Define scan region corners
    if event == cv2.EVENT_RBUTTONDOWN:
        if len(scan_region_pixels) < 2:
            scan_region_pixels.append((x, y))
            print(f"📍 Scan corner {len(scan_region_pixels)}: pixel ({x}, {y})")
            rx, ry = robot.pixel_to_robot(x, y)
            print(f"   → Robot: ({rx:.1f}, {ry:.1f})")
        else:
            scan_region_pixels = [(x, y)]
            print(f"📍 New scan region - corner 1: pixel ({x}, {y})")
    
    # LEFT CLICK = Select object
    elif event == cv2.EVENT_LBUTTONDOWN and height_map_ready:
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    cx, cy = selected_grasp['center']
                    rx, ry = robot.pixel_to_robot(cx, cy)
                    z_from_map = scanner.get_height_at(rx, ry)
                    print(f"\n📦 Object: W={selected_grasp['width_mm']:.1f}mm")
                    print(f"   Position: ({rx:.1f}, {ry:.1f})")
                    print(f"   Z from map: {z_from_map:.1f}mm" if z_from_map else "   Z from map: N/A")
                break

def draw_ui(frame):
    display = frame.copy()
    
    # Draw scan region
    if len(scan_region_pixels) >= 1:
        cv2.circle(display, scan_region_pixels[0], 8, (0, 255, 255), -1)
    if len(scan_region_pixels) == 2:
        cv2.rectangle(display, scan_region_pixels[0], scan_region_pixels[1], (0, 255, 255), 2)
        cv2.putText(display, "Scan Region", (scan_region_pixels[0][0], scan_region_pixels[0][1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    
    # Draw objects
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0, 0, 255) if is_sel else (0, 255, 0)
        cv2.rectangle(display, (x, y), (x+w, y+h), color, 2)
    
    # Draw grasp
    if selected_grasp:
        cx, cy = selected_grasp['center']
        angle = selected_grasp['camera_angle']
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(display, (cx-dx, cy-dy), (cx+dx, cy+dy), (0, 0, 255), 3)
        cv2.circle(display, (cx, cy), 5, (0, 0, 255), -1)
    
    # Status bar
    cv2.rectangle(display, (0, 0), (640, 35), (30, 30, 30), -1)
    status = "MAP READY" if height_map_ready else "NO MAP"
    cv2.putText(display, f"v13.2 | {status} | Obj:{len(detected_objects)} | S=Scan | SPACE=Pick",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    if selected_grasp:
        cv2.putText(display, f"[W={selected_grasp['width_mm']:.1f}mm - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    return display

In [11]:
def pick_with_height_map(obj, grasp):
    """v13.2: Pick using height from pre-scanned map"""
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    
    gripper_x, gripper_y = robot.pixel_to_robot(cx, cy)
    
    # Get Z from height map
    z_from_map = scanner.get_height_at(gripper_x, gripper_y)
    
    if z_from_map is None:
        print("❌ Position outside height map! Aborting.")
        return
    
    # Apply height correction
    estimated_height = max(0, Z_FLOOR - z_from_map + (Z_MEASURE - Z_FLOOR))
    height_correction = estimated_height * HEIGHT_CORRECTION_FACTOR
    z_grasp = z_from_map - height_correction
    z_grasp = max(Z_FLOOR, z_grasp)
    
    print(f"\n🤖 Pick v13.2 (Height Map): W={grip_w:.1f}mm")
    print(f"   Position: ({gripper_x:.1f}, {gripper_y:.1f})")
    print(f"   Z from map: {z_from_map:.1f} → Z grasp: {z_grasp:.1f}")
    
    robot_r = ROBOT_R_OFFSET
    
    # Execute pick sequence (no LIDAR measurement needed!)
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 1)
    
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(2)
    
    print("🎯 Moving above object...")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, robot_r, 2)
    
    final_r = robot.camera_angle_to_robot_r(camera_angle)
    print(f"🔄 Rotating to R={final_r:.1f}°...")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, final_r, 2)
    
    print(f"⬇️ Going down to Z={z_grasp:.1f}...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_grasp, final_r, 2)
    
    gripper.grip_object(grip_w - 8.5)
    time.sleep(4)
    
    z_lift = z_grasp + 50
    print(f"⬆️ Lifting to Z={z_lift:.1f}...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_lift, final_r, 2)
    
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    gripper.release()
    time.sleep(2)
    
    drop_x, drop_y, drop_z, drop_r = DROP_POS
    robot.move_to_and_wait(drop_x, drop_y, 150, drop_r, 2)
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    robot.home()
    print("✅ Complete!")

In [12]:
# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('v13.2 Height Map')
cv2.setMouseCallback('v13.2 Height Map', mouse_callback)

print("="*60)
print("🗺️ v13.2 3D Height Map System")
print("Right-Click: Define scan region (2 corners)")
print("S: Start scan | M: Toggle heatmap | P: Plot 3D")
print("Left-Click: Select object | SPACE: Pick")
print("R: Reset | H: Home | Q: Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected_objects = detector.detect(frame)
    
    # Overlay heatmap if enabled
    if show_heatmap and height_map_ready:
        heatmap = scanner.get_height_heatmap(frame.shape)
        if heatmap is not None:
            frame = cv2.addWeighted(frame, 0.6, heatmap, 0.4, 0)
    
    display = draw_ui(frame)
    cv2.imshow('v13.2 Height Map', display)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    
    elif key == ord('s') and len(scan_region_pixels) == 2:
        # Start scan
        p1, p2 = scan_region_pixels
        r1 = robot.pixel_to_robot(p1[0], p1[1])
        r2 = robot.pixel_to_robot(p2[0], p2[1])
        
        x_min, x_max = min(r1[0], r2[0]), max(r1[0], r2[0])
        y_min, y_max = min(r1[1], r2[1]), max(r1[1], r2[1])
        
        scanner.scan_region(x_min, x_max, y_min, y_max, Z_MEASURE)
        height_map_ready = True
        robot.home()
    
    elif key == ord('m'):
        show_heatmap = not show_heatmap
        print(f"Heatmap: {'ON' if show_heatmap else 'OFF'}")
    
    elif key == ord('p') and height_map_ready:
        scanner.plot_3d()
    
    elif key == ord('r'):
        scan_region_pixels = []
        selected_object = None
        selected_grasp = None
        current_grasps = []
        print("🔄 Reset")
    
    elif key == ord('h'):
        robot.home()
    
    elif key == ord(' ') and selected_object and selected_grasp and height_map_ready:
        pick_with_height_map(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()
gripper.disconnect()

🗺️ v13.2 3D Height Map System
Right-Click: Define scan region (2 corners)
S: Start scan | M: Toggle heatmap | P: Plot 3D
Left-Click: Select object | SPACE: Pick
R: Reset | H: Home | Q: Quit
📍 Scan corner 1: pixel (293, 119)
   → Robot: (47.0, 55.0)
📍 Scan corner 2: pixel (420, 425)
   → Robot: (-39.6, 12.5)

🗺️ Starting 3D Height Map Scan
   Region: X[-40, 47] Y[12, 55]
   Grid: 5mm | Passes: 3 | Speed: 10%

📏 Pass 1/3
   Line 1/10 done (3%)
   Line 2/10 done (7%)
   Line 3/10 done (10%)
   Line 4/10 done (13%)
   Line 5/10 done (17%)
   Line 6/10 done (20%)
   Line 7/10 done (23%)
   Line 8/10 done (27%)
   Line 9/10 done (30%)
   Line 10/10 done (33%)

📏 Pass 2/3
   Line 1/10 done (37%)
   Line 2/10 done (40%)
   Line 3/10 done (43%)
   Line 4/10 done (47%)
   Line 5/10 done (50%)
   Line 6/10 done (53%)
   Line 7/10 done (57%)
   Line 8/10 done (60%)
   Line 9/10 done (63%)
   Line 10/10 done (67%)

📏 Pass 3/3
   Line 1/10 done (70%)
   Line 2/10 done (73%)
   Line 3/10 done (77%)
 

: 